## Datashading 2010 US Census data with HoloViews/GeoViews

The regular [census.ipynb](https://anaconda.org/jbednar/census) notebook from the [datashader](https://github.com/bokeh/datashader) library shows a variety of analyses of the 2010 Census data on population and race.  Here we show a simpler, standalone example of the basic process of making an interactive datashader plot overlaid on a map.

### Set up and load data

In [ ]:
import dask.dataframe as dd
import holoviews as hv
import geoviews as gv
import datashader as ds

from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade, dynspread
hv.notebook_extension('bokeh')

df = dd.io.parquet.read_parquet('data/census.snappy.parq')
%time df=df.persist()

### Interactive plot of population density with race as colors, overlaid on a geographic map

HoloViews will happily pass on any size of dataset for the browser to display using Bokeh, but the browser would crash if you were to try to visualize the 300-million-row ``points`` object below. Luckily, HoloViews makes it simple to add a dynamic call to datashader that aggregates the data into a new image on any zoom or pan, to ensure that big data can safely be explored interactively.  Of course, this dynamic updating requires a live Python server running; with a static rendering like anaconda.org you'll only see the initial image drawn.

In [ ]:
%%opts Overlay [width=900 height=525 xaxis=None yaxis=None]
geourl = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
geomap = gv.WMTS(WMTSTileSource(url=geourl))
points = gv.Points(df, kdims=['easting', 'northing'], vdims=['race'])
color_key = {'w':'blue',  'b':'green', 'a':'red',   'h':'orange',   'o':'saddlebrown'}

race = dynspread(datashade(points, color_key=color_key, element_type=hv.Image, aggregator=ds.count_cat('race')))
geomap * race